In [1]:
import json
import os
import typing

PARENT_DIR = os.path.dirname(os.getcwd())

import apache_beam as beam
import apache_beam.runners.interactive.interactive_beam as ib
from apache_beam.runners.interactive import interactive_runner

In [2]:
class EventLog(typing.NamedTuple):
    ip: str
    id: str
    lat: float
    lng: float
    user_agent: str
    age_bracket: str
    opted_into_marketing: bool
    http_request: str
    http_response: int
    file_size_bytes: int
    event_datetime: str
    event_ts: int

beam.coders.registry.register_coder(EventLog, beam.coders.RowCoder)

def parse_json(element: str):
    row = json.loads(element)
    # lat/lng sometimes empty string
    if not row["lat"] or not row["lng"]:
        row = {**row, **{"lat": -1, "lng": -1}}
    return EventLog(**row)

In [3]:
p = beam.Pipeline(interactive_runner.InteractiveRunner())
events = (
    p
    | "Read from files"
    >> beam.io.ReadFromText(
        file_pattern=os.path.join(os.path.join(PARENT_DIR, "inputs", "*.out"))
    )
    | "Parse elements" >> beam.Map(parse_json).with_output_types(EventLog)
)

In [4]:
%load_ext apache_beam.runners.interactive.sql.beam_sql_magics

In [5]:
%%beam_sql -o output
SELECT id, COUNT(*) AS page_views, SUM(file_size_bytes) AS total_bytes, MAX(file_size_bytes) AS max_bytes, MIN(file_size_bytes) AS min_bytes FROM events GROUP BY id

2.53.0: Pulling from apache/beam_java11_sdk
Digest: sha256:4f90eceef156cdab47136c978b873c6dc84bb9812a65fbbc515e9cf0071ffd5e
Status: Image is up to date for apache/beam_java11_sdk:2.53.0
docker.io/apache/beam_java11_sdk:2.53.0
1645ce3cca7d27742385d2dd5aaaa4ae32e725d1d8820b4f82a63c99297b75db


<PCollection[SqlTransform(beam:external:java:sql:v1).output] at 0x7ffb83fe0fa0>